# Assignment 2 Computer Vision Part 2


In [2]:
import torch
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms as T

In [3]:
## getting data
transform = T.Compose(
    [T.ToTensor(),
     T.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
resnet18 = models.resnet18(pretrained=False)
print(resnet18)

# Input size of resnet18 -> 224 by 224